In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!unzip /content/aviachipta-narxini-bashorat-qilish.zip

Archive:  /content/aviachipta-narxini-bashorat-qilish.zip
  inflating: sample_solution.csv     
  inflating: test_data.csv           
  inflating: train_data.csv          


In [135]:
df=pd.read_csv("train_data.csv").drop('id',axis=1)
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [63]:
df=df.sort_values('airline')

In [64]:
df.shape

(20000, 11)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 18476 to 19999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           20000 non-null  object 
 1   flight            20000 non-null  object 
 2   source_city       20000 non-null  object 
 3   departure_time    20000 non-null  object 
 4   stops             20000 non-null  object 
 5   arrival_time      20000 non-null  object 
 6   destination_city  20000 non-null  object 
 7   class             20000 non-null  object 
 8   duration          20000 non-null  float64
 9   days_left         20000 non-null  int64  
 10  price             20000 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 1.8+ MB


In [10]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [66]:
df.airline.value_counts()

Vistara      8535
Air_India    5371
Indigo       2924
GO_FIRST     1508
AirAsia      1056
SpiceJet      606
Name: airline, dtype: int64

In [75]:
def labeler(dataset,dnCol):
  dataset2=dataset.drop(dnCol,axis=1)
  dict={}
  for col in dataset2.columns:
      if dataset2[col].dtype=='O':
        dict[col]=encoder.fit_transform(dataset2[col])
      else:
        dict[col]=dataset2[col]
  newdf=pd.DataFrame(dict)
  return pd.concat([newdf,dataset[dnCol]],axis=1)

df=labeler(df,'price')
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
18476,0,987,1,4,1,3,2,1,17.00,13,7353
16762,0,1004,5,1,0,2,2,1,13.42,9,31799
12038,0,953,1,5,2,5,0,1,1.50,17,1548
19585,0,1009,2,2,1,4,1,1,16.33,26,1998
14862,0,1001,4,0,0,3,5,1,11.08,19,3727


In [53]:
df.corrwith(df.price)

airline             0.241193
flight              0.303288
source_city         0.013570
departure_time      0.058520
stops              -0.201309
arrival_time        0.046801
destination_city    0.008747
class              -0.937868
duration            0.213158
days_left          -0.102545
price               1.000000
dtype: float64

In [121]:
X=df.drop('price',axis=1)
y=df[['price']]

In [122]:
from sklearn.model_selection import train_test_split
train_set,test_set,train_setlb,test_setlb=train_test_split(X,y,test_size=0.1,random_state=50)


In [142]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [148]:
train_set_scaled=scaler.fit_transform(train_set)
test_set_scaled=scaler.fit_transform(test_set)

In [149]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error

LinearModel


In [163]:
model=LinearRegression()
model.fit(train_set_scaled,train_setlb)
pred=model.predict(test_set_scaled)
print('mae=',mean_absolute_error(test_setlb,pred))
print('rmse=',mean_squared_error(test_setlb,pred)**0.5)
pd.DataFrame({'real':test_setlb.to_numpy().reshape(-1),'precision':pred.reshape(-1)}).sample(5)


mae= 4631.710487785922
rmse= 6836.958902782638


,real,precision
1579,1998,-606.449080
1052,75023,57565.981721
1355,5206,6721.094329
1351,1890,-1700.876632
330,39321,52710.032933


DeciosnTreeRegression

In [164]:
model=DecisionTreeRegressor()
model.fit(train_set_scaled,train_setlb)
pred=model.predict(test_set_scaled)
print('mae=',mean_absolute_error(test_setlb,pred))
print('rmse=',mean_squared_error(test_setlb,pred)**0.5)
pd.DataFrame({'real':test_setlb.to_numpy().reshape(-1),'precision':pred.reshape(-1)}).sample(5)


mae= 2890.66575
rmse= 5743.879552108052


,real,precision
500,7412,11346.0
829,2074,2124.0
931,3220,2111.0
1626,8803,6476.0
416,2098,2156.0


RandomForest

In [172]:
model=RandomForestRegressor()
model.fit(train_set_scaled,train_setlb.to_numpy().reshape(-1))
pred=model.predict(test_set_scaled)
print('mae=',mean_absolute_error(test_setlb,pred))
print('rmse=',mean_squared_error(test_setlb,pred)**0.5)
pd.DataFrame({'real':test_setlb.to_numpy().reshape(-1),'precision':pred.reshape(-1)}).sample(5)


mae= 2325.9923475
rmse= 4284.051573558374


,real,precision
701,56702,58940.78
1318,6489,7179.11
1654,1694,2816.63
1236,5322,5385.02
420,42457,40541.17


In [175]:
out_set=pd.read_csv('/content/test_data.csv').drop('id',axis=1)
out_set.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,Air_India,AI-765,Kolkata,Evening,one,Night,Delhi,Business,28.25,2
1,Vistara,UK-747,Delhi,Early_Morning,one,Night,Mumbai,Business,13.83,34
2,Air_India,AI-570,Mumbai,Early_Morning,zero,Early_Morning,Chennai,Business,2.00,30
3,AirAsia,I5-974,Hyderabad,Night,one,Late_Night,Delhi,Economy,5.17,26
4,Air_India,AI-770,Kolkata,Night,one,Afternoon,Mumbai,Economy,16.33,35


In [179]:
def labeler2(dataset):

  dict={}
  for col in dataset.columns:
      if dataset[col].dtype=='O':
        dict[col]=encoder.fit_transform(dataset[col])
      else:
        dict[col]=dataset[col]
  newdf=pd.DataFrame(dict)
  return newdf



out_set=labeler2(out_set,)
out_set

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,1,452,4,2,0,5,2,0,28.25,2
1,5,834,2,1,0,5,5,0,13.83,34
2,1,401,5,1,2,1,1,0,2.00,30
3,0,723,3,5,0,3,2,1,5.17,26
4,1,456,4,5,0,0,5,1,16.33,35
...,...,...,...,...,...,...,...,...,...,...
4995,1,455,4,0,0,4,0,0,17.42,15
4996,3,221,4,4,2,0,5,1,3.00,40
4997,1,344,4,4,0,5,5,0,11.17,37
4998,1,432,5,1,0,5,3,0,13.33,38


In [180]:
out_set_scaled=scaler.fit_transform(out_set)


In [184]:
out_set_labels=model.predict(out_set_scaled)
out_set['Prediction_price']=out_set_labels

In [183]:
out_set

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,Prediction_price
0,1,452,4,2,0,5,2,0,28.25,2,56837.48
1,5,834,2,1,0,5,5,0,13.83,34,63320.18
2,1,401,5,1,2,1,1,0,2.00,30,23403.80
3,0,723,3,5,0,3,2,1,5.17,26,2676.45
4,1,456,4,5,0,0,5,1,16.33,35,5611.49
...,...,...,...,...,...,...,...,...,...,...,...
4995,1,455,4,0,0,4,0,0,17.42,15,59802.69
4996,3,221,4,4,2,0,5,1,3.00,40,4342.98
4997,1,344,4,4,0,5,5,0,11.17,37,52520.48
4998,1,432,5,1,0,5,3,0,13.33,38,43853.31


In [187]:
import joblib
joblib.dump(model,'RF_model_AirTickets.jbl')
#downloaded_model=joblib.load('/content/RF_model_AirTickets.jbl')

['RF_model_AirTickets.jbl']

In [ ]:
from sklearn.model_selection import cross_val_score
scores=cross_val_score(RandomForestRegressor(),scaler.fit_transform(X),y,scoring="neg_mean_squared_error",cv=5)

In [206]:
np.sqrt(-scores)

array([3243.8641429 , 2828.49150908, 2896.92291267, 4734.79389251,
       4563.44923932])